# Library and Data Import

In [9]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Fetch the dataset
car_dataset = fetch_ucirepo(id=19)

# Data unpacking
X = car_dataset.data.features
y = car_dataset.data.targets

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,random_state=12)

# variable information 
print(car_dataset.variables) 

       name     role         type demographic  \
0    buying  Feature  Categorical        None   
1     maint  Feature  Categorical        None   
2     doors  Feature  Categorical        None   
3   persons  Feature  Categorical        None   
4  lug_boot  Feature  Categorical        None   
5    safety  Feature  Categorical        None   
6     class   Target  Categorical        None   

                                         description units missing_values  
0                                       buying price  None             no  
1                           price of the maintenance  None             no  
2                                    number of doors  None             no  
3              capacity in terms of persons to carry  None             no  
4                           the size of luggage boot  None             no  
5                        estimated safety of the car  None             no  
6  evaulation level (unacceptable, acceptable, go...  None             no  

In [ ]:
car_dataset.metadata

## Data Exploration

In [ ]:
print(type(X))
print(type(y))

In [ ]:
print(f"{X.describe()} \n")

print(X.sample(20))

In [10]:
df = pd.concat([X,y],axis=1)
df.sample(10)

,buying,maint,doors,persons,lug_boot,safety,class
1514,low,med,2,2,small,high,unacc
1501,low,high,5more,4,big,med,acc
1696,low,low,4,more,med,med,good
203,vhigh,high,5more,4,med,high,unacc
991,med,high,2,more,small,med,unacc
522,high,vhigh,5more,4,small,low,unacc
961,med,vhigh,5more,4,big,med,acc
971,med,vhigh,5more,more,big,high,acc
1165,med,med,5more,2,med,med,unacc
1510,low,high,5more,more,big,med,acc


In [11]:
y['class'].unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [41]:
df_train = pd.concat([X_train,y_train],axis=1)
print(df_train.shape)

df_test = pd.concat([X_test,y_test],axis=1)
print(df_test.shape)

(1382, 7)
(346, 7)


# Naive Bayes algorithm - Yappington Academy

In a Naives Bayes classifier, one can learn the probability that an outcome happens given the conditions of the feature. Taken the tennis example, when given that the day is sunny (S), temperature is cool (C), humidity is high (H), and wind is strong (St), the desired output is the probability that players come to the court. Using Naive Bayes assumption, one can calculate this probability as 

$$p(Play\_Tennis = Y|Sunny,Cool,High,Strong) = p(Y)\times p(S|Y)\times p(C|Y)\times p(H|Y)\times p(St|Y)$$

The prior probability of Play_Tennis = Y is calculated as the number of yes occurences over the total number of samples. The conditional probabilities (such as $p(S|Y)$) is determined from the given dataset as the number of occurences where Play_Tennis is Yes and Outlook is Sunny over the total number of occurences where Play_Tennis is Yes. 
$$p(Outlook = Sunny|Play\_Tennis=Yes) = \frac{\text{\# S and Y}}{\text{\# Y}}$$


In the car evaluation example, there are three outcomes - unacceptable, acceptable, good, and vgood. The categories of the features were explored in the previous step and reported here
* buying: The buying price has 4 categories - low, med, high, and vhigh
* maint: The maintenance price has the similar 4 categories
* doors: The number of doors has 4 categories - 2, 3, 4, and 5more
* person: The number of passenger capacity has 3 categories - 2, 4, and more
* lug_boot: The size of luggage boot has 3 categories - small, med, and big
* safety: The estimated car safety has 3 categories - low, med, and high

In this example, the relative probability of an instance belong to a class is found as
$$p(class | \text{feature 1, feature 2, ... feature n}) = p(class)\prod_{i=1}^n p(\text{feature i}|class)$$

The class with the highest probability is the class that the instance belongs to. To implement the Naive Bayes classifier, I will then need to be able to:
* Find the prior probabilities of each class happening. This is simple since I just need to find the number of instances that belongs to a class and the size of the training dataset
* Find the conditional probability of p(feature i | class). This is a bit more involved since I will need to consider each feature value given that the instance belongs to a class -

# Functional implementation of Naive Bayes

## Functions for determining prior and conditional probabilities 

In [13]:
def count_occurrences(df, conditions):
    
    """
    Counts the number of occurrences in a DataFrame that match given conditions.

    Parameters
    - df: DataFrame to count occurrences from.
    - conditions: Dictionary specifying conditions (e.g., {'column_name': 'value'}).

    Returns:
    - int: The count of occurrences matching the conditions.
    """
    # Start with the entire DataFrame and filter down based on conditions
    filtered_df = df
    for column, value in conditions.items():
        filtered_df = filtered_df[filtered_df[column] == value]
    return len(filtered_df)

print(f"Number of occurences where: \n"
      f"- Buying price is VHIGH: {count_occurrences(df,{'buying':'vhigh'})}\n"
      f"- Class is VGOOD: {count_occurrences(df,{'class':'vgood'})}\n"
      f"- Buying price is VHIGH and class is GOOD: {count_occurrences(df, {'buying': 'vhigh','class':'good'})} \n"
      f"- Buying price is HIGH and class is UNACC: {count_occurrences(df, {'buying': 'vhigh','class':'unacc'})} \n"
      f"- Safety is HIGH and class is UNACC: {count_occurrences(df, {'safety': 'vhigh','class':'unacc'})} \n"
      f"- Doors is 5MORE and class is GOOD: {count_occurrences(df, {'doors': '5more','class':'good'})} \n"
      f"- Maintenance price is HIGH and class is GOOD: {count_occurrences(df, {'maint': 'high','class':'good'})}")

Number of occurences where: 
- Buying price is VHIGH: 432
- Class is VGOOD: 65
- Buying price is VHIGH and class is GOOD: 0 
- Buying price is HIGH and class is UNACC: 360 
- Safety is HIGH and class is UNACC: 0 
- Doors is 5MORE and class is GOOD: 18 
- Maintenance price is HIGH and class is GOOD: 0


In [ ]:
df.columns

In [ ]:
def find_prior_prob(df, condition):
    ''' 
    This function finds the prior probability P(a=A) by determining the number of samples in the entire dataset and the number of occurences of A

    Parameters:
    - df: DataFrame to count occurrences from.
    - conditions: Dictionary specifying the condition or the value of event A (e.g. {'buying'}:{'high'})

    Returns:
    - float: The prior probability that even A happens 
    '''
    num_instances = len(df)
    num_cond = count_occurrences(df, condition)
    
    prior_prob = num_cond / num_instances

    return prior_prob

def find_prior_probs(df, label_column):
    '''
    This function finds the prior probability of each class in the dataframe and report them in a dictionary

    Parameters:
    - df (pd dataframe): DataFrame from which the probability is determined
    - label_colum (str): name of the class/label column

    Returns:
    - class_prior (dict): the dictionary containing class (key) and their corresponding prior probabilities (value)
    '''
    classes = df[label_column].unique()                 # List of the classes
    num_instances = len(df)
    class_prior_probs = {}

    for class_value in classes:
        class_count = df[df[label_column]==class_value].shape[0]
        class_prior_probs[str(class_value)] = class_count / num_instances

    return class_prior_probs

def find_cond_prob(df, prior_condition, condition):
    ''' 
    This function finds the conditional probability P(a=A|b=B) by first finding the number of B occurences in the data, then find the occurences of A given that B occured
    
    Parameters:
    - df: DataFrame to count occurrences from.
    - prior_condition: Dictionary specifying the condition or the value of event B (e.g. {'class'}:{'good'})
    - conditions: Dictionary specifying the condition or the value of event A (e.g. {'buying'}:{'high'})


    Returns:
    - float: The conditional probability P(A|B)
    '''
    # Filter down the dataframe to only include the occurences that satisfy the prior condition
    for column, value in prior_condition.items():
        filtered_df = df[df[column] ==  value]
    num_instances = len(filtered_df)

    # Count the number of occurences that satify the condition, given that the prior condition is already satisfied
    # (In other words, count the occurences that satisfy the condition from the filtered dataframe)
    num_cond = count_occurrences(filtered_df,condition)
    
    cond_prob = num_cond/num_instances

    return cond_prob 

def find_cond_probs(df, label_column):
    classes = df[label_column].unique()

    feature_cond_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    '''This is a nested dictionary of the following structure
        - Level 1 - Class Values: This level has the class values and the next dictionary level as the key-value pair
            - Level 2 - Feature Names: This level of dictionary has the feature name and the next dictionary level as the key-value pair
                - Level 3 - Feature Values: This level of dictionary has the feature value and the conditional probabilities P(feature == feature value | class == class value) as the key-value pair 
    '''

    # Considering the conditional probability feature by feature
    for feature in df.columns:
        # Skip this step if the feature is the label column of the dataframe
        if feature == label_column:
            pass
        
        # Iterate over each class c of the dataframe to find the conditional probabilities of each feature P(feature = value | class)
        for c in classes: 
            df_subset = df[df[label_column]==c]                         # Subset of the dataframe where class has the value c
            feature_count = df_subset[feature].value_counts()           # Get a series with the number of instances of each feature value in the subset

            # Iterate over all the feature values and calculate the conditional probabilities of each value 
            for value, count in feature_count.items():                  
                feature_cond_probs[c][feature][value] = count / len(df_subset)

    return feature_cond_probs

## Evaluating specific conditional probabilities
This section is to test the find_prior_prob() and find_cond_prob() functions, which determine the prior and condition probabilities given a single specific condition

In [16]:
print(f"The prior probability of the class LOW: {find_prior_prob(df_train, {'class':'unacc'})}")

print(f"The probability that buying price is LOW, given that the class is VGOOD is: {find_cond_prob(df_train, {'class':'vgood'}, {'buying':'low'})} \n"
      f"The probability that the class is VGOOD, given that buying price is LOW is: {find_cond_prob(df_train, {'buying':'low'}, {'class':'vgood'})}")

print(f" => From this finding, one can conclude that if the car is VGOOD, the buying price of the car is most likely LOW.\n" 
      f"    However, having a low price does not guarantee that the car is classified as VGOOD since there are other deterrent criteria such as high maintenance price or low safety\n")

print(f"The conditional probability that buying price is LOW, given that the class is VGOOD is: {find_cond_prob(df_train, {'class':'vgood'}, {'buying':'low'})} \n"
      f"The conditional probability that buying price is MED, given that the class is VGOOD is: {find_cond_prob(df_train, {'class':'vgood'}, {'buying':'med'})} \n"
      f"The conditional probability that buying price is HIGH, given that the class is VGOOD is: {find_cond_prob(df_train, {'class':'vgood'}, {'buying':'high'})} \n"
      f"The conditional probability that buying price is VHIGH, given that the class is VGOOD is: {find_cond_prob(df_train, {'class':'vgood'}, {'buying':'vhigh'})}")

print(" => From this finding, one can conclude that if the car is VGOOD, the buying price of the car can only be LOW or MED and never HIGH or VHIGH \n")

print(f"The conditional probability that buying price is LOW, given that the class is UNACC is: {find_cond_prob(df_train, {'class':'unacc'}, {'buying':'low'})} \n"
      f"The conditional probability that buying price is MED, given that the class is UNACC is: {find_cond_prob(df_train, {'class':'unacc'}, {'buying':'med'})} \n"
      f"The conditional probability that buying price is HIGH, given that the class is UNACC is: {find_cond_prob(df_train, {'class':'unacc'}, {'buying':'high'})} \n"
      f"The conditional probability that buying price is VHIGH, given that the class is UNACC is: {find_cond_prob(df_train, {'class':'unacc'}, {'buying':'vhigh'})}")

print(" => From this finding, one can conclude that if the car is UNACC, the buying price can be of any value but more likely HIGH or VHIGH.")

The prior probability of the class LOW: 0.6982633863965267
The probability that buying price is LOW, given that the class is VGOOD is: 0.6071428571428571 
The probability that the class is VGOOD, given that buying price is LOW is: 0.09742120343839542
 => From this finding, one can conclude that if the car is VGOOD, the buying price of the car is most likely LOW.
    However, having a low price does not guarantee that the car is classified as VGOOD since there are other deterrent criteria such as high maintenance price or low safety

The conditional probability that buying price is LOW, given that the class is VGOOD is: 0.6071428571428571 
The conditional probability that buying price is MED, given that the class is VGOOD is: 0.39285714285714285 
The conditional probability that buying price is HIGH, given that the class is VGOOD is: 0.0 
The conditional probability that buying price is VHIGH, given that the class is VGOOD is: 0.0
 => From this finding, one can conclude that if the car 

## Determine the conditional probabilities of all cases
This section shows the result of the functions find_prior_probs() and find_cond_probs(), which determine the prior and conditional probabilities for all classes and all probability 
$$P(\text{feature values}|\text{class values})$$

In [29]:
prior_probs = find_prior_probs(df_train,'class')

for key, value in prior_probs.items():
    print(f"The prior probability of class = {key} is {value*100:.3f}%")

The prior probability of class = unacc is 69.826%
The prior probability of class = acc is 22.431%
The prior probability of class = vgood is 4.052%
The prior probability of class = good is 3.690%


In [34]:
cond_probs = find_cond_probs(df_train,label_column='class')
cond_probs

defaultdict(<function __main__.find_cond_probs.<locals>.<lambda>()>,
            {'unacc': defaultdict(<function __main__.find_cond_probs.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'buying': defaultdict(float,
                                      {'vhigh': 0.2994818652849741,
                                       'high': 0.26632124352331604,
                                       'low': 0.21761658031088082,
                                       'med': 0.21658031088082902}),
                          'maint': defaultdict(float,
                                      {'vhigh': 0.3067357512953368,
                                       'high': 0.26528497409326424,
                                       'med': 0.21865284974093263,
                                       'low': 0.20932642487046632}),
                          'doors': defaultdict(float,
                                      {'2': 0.2549222797927461,
                                       '5more': 0.2

In [ ]:
def naive_bayes_classifier(prior_probs, cond_probs, instances):
    '''
    This function is used for classifying an instance

    Parameters
    - cond_probs: the 3-tier dictionary structure returned by the find_cond_probs() function. This structure represents the conditional probabilities P(features | classes) and has the structure of [class value][feature][feature value]
    - instance: a dictionary representing a data point ()
    '''

    # Get the list of classes (from the cond_probs structures)
    classes = []
    for item in cond_probs.items():
        classes.append(item[0])

    # List of empty dictionaries to store the probabilities of the instance belong to each class
    class_probs = [{}] * len(instances)                                        
    
    for index, instance in enumerate(instances):
        # Iterate over each class to find the prior probability 
        for c in classes:
            class_probs[index][c] = prior_probs[c]                     # p(class==c)

            # Iterate over each feature in the instance to find the accumulative of conditional probabilties
            for feature,value in instance.items():
                class_probs[index][c] *= max(cond_probs[c][feature][value],1e-6)       # In case of zero probability, use a small number
        
        class_probs[index] = max(class_probs[index], key=class_probs[index].get)

    return class_probs

## Test the Naive Bayes Classifier

In [195]:
# Sample some random test_instances and drop the index to later concatenate with the predicted label
test_instances = df_test.sample(25).reset_index(drop=True)
test_instance_features = test_instances.drop('class',axis =1).to_dict(orient='records')
test_instance_labels = test_instances['class'].values


prediction = naive_bayes_classifier(prior_probs=prior_probs,
                                    cond_probs=cond_probs,
                                    instances=test_instance_features)

prediction_df = pd.DataFrame(prediction, columns=["Predicted_Class"])
prediction_result = pd.concat([test_instances,prediction_df],axis=1)

print(prediction_result)

   buying  maint  doors persons lug_boot safety  class Predicted_Class
0   vhigh   high      3    more      med    med  unacc           unacc
1     med   high      3    more      med    med    acc             acc
2     low    low  5more       2      med    med  unacc           unacc
3   vhigh  vhigh      3       2      big    low  unacc           unacc
4   vhigh    low      2       2    small   high  unacc           unacc
5     med   high      3       4      big    low  unacc           unacc
6     low  vhigh  5more       2      big    low  unacc           unacc
7     low   high      2       2    small    low  unacc           unacc
8    high    med      4       4    small    med  unacc             acc
9     low  vhigh      3       4      big    med    acc           unacc
10   high    low      2    more      big    low  unacc           unacc
11  vhigh    low  5more       2    small    low  unacc           unacc
12    low    med      2       2      big    med  unacc           unacc
13   h

In [177]:
for key,value in test_instance_features[1].items():
    print(key,value)

buying vhigh
maint med
doors 2
persons more
lug_boot big
safety high


# Naives Bayes Classifier - OOP approach

As can be seen in the functional implementation of the Naives Bayes, one need to run the find_prior_probs() and find_cond_probs() functions first before passing them into the final naive_baye_classifier() to predict an instance. This approach can be deemed as unwieldy for downstream application. 

Inspired by the approach that machine learning libraries such as Tensorflow adopted, the following section shows the OOP implementation of Naive Bayes. In this approach, I will define a blueprint for a NaiveBayesClassifier() object, which stores attributes such as the classes, prior probabilities, and conditional probabilities learned from running a fit() method. Afterward, a predict() method can be run on a list of testing instances to provide prediction 